# Prerequisites

In [1]:
%%capture
pip install requests docling transformers ipywidgets minio

In [2]:
%%capture
pip install llama-index-core llama-index-readers-docling llama-index-node-parser-docling llama-index-embeddings-huggingface llama-index-llms-huggingface-api llama-index-vector-stores-milvus llama-index-readers-file python-dotenv;

In [3]:
## Docling and chunker for text parsing
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker
## embeddings generator
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

# Docling ingestion

## Define global variable

In [4]:
## Source for the documents for docling parsing of chunks
source_file = "https://docs.redhat.com/en/documentation/red_hat_openshift_ai_self-managed/2.16/pdf/monitoring_data_science_models/Red_Hat_OpenShift_AI_Self-Managed-2.16-Monitoring_data_science_models-en-US.pdf"  # document per local path or URL

## 0) Ingest documentation - Test

In [5]:
## Define Converter
converter = DocumentConverter()

## Parse document from source and store in variable "document"
document = converter.convert(source_file)


## Print to check everything is working
# print(document.document.export_to_markdown())
lines = document.document.export_to_markdown().splitlines()
for line in lines[:10]:
    print(line)

<!-- image -->

## Red Hat OpenShift AI Self-Managed 2.16

## Monitoring data science models

Monitor your OpenShift AI models for fairness

## Red Hat OpenShift AI Self-Managed 2.16 Monitoring data science models



## 1) Chunking the downloaded document using Docling

In [6]:
## Parse document from source and store in variable "document"
converted_source_file = DocumentConverter().convert(source_file)
document = converted_source_file.document

## Create chubker and chuck document
chunker = HybridChunker(tokenizer="BAAI/bge-small-en-v1.5")  # set tokenizer as needed
chunk_iter = chunker.chunk(document)

## Create chunk_list with the parts of the document
chunk_list = list(chunk_iter)

#for i,chunk in enumerate(chunk_iter):
    #print(i)
    #print(chunk)

## Print to check everything is working
# print(chunk_list[0])
print(chunk_list[0].text)

Token indices sequence length is longer than the specified maximum sequence length for this model (925 > 512). Running this sequence through the model will result in indexing errors


Monitor your OpenShift AI models for fairness
Last Updated: 2024-12-11


## 2) Create embeddings

In [7]:
## Defined embedding model and import it
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

## Define Empty Vector Array
vectors = []
## Populate Vector Array

for i, chunk in enumerate(chunk_list):
    vectors.append({
        "id": i, 
        "vector": embedding_model.get_text_embedding(chunk.text) , 
        "text": chunk.text
    })

## Print to check everything is working
print(f'Length of the Vector: {len(vectors)}' )

Length of the Vector: 112


## Extra! Ingesting Multiple files

In [29]:
def get_version_from_url(url):
    separate = url.split("-")
    separate = separate[2]
    
    return separate

In [30]:
from urllib.parse import urlparse
def get_file_name_from_url(url):
    # Parse the URL to extract the path
    parsed_url = urlparse(url)
    # Extract the file name from the path
    file_name = parsed_url.path.split('/')[-1]
    
    return file_name

In [31]:
def get_metadata_from_filename(filename):
    metadata = filename.split("-")
    return {
            "product_name": metadata[0],
            "version": metadata[2],
            "section": metadata[3],
            "language": metadata[4]
        }

In [41]:
## Create the converter 
converter = DocumentConverter()
## Create chunker to chunk
chunker = HybridChunker(tokenizer="BAAI/bge-small-en-v1.5")  # set tokenizer as needed
## Defined embedding model and import it
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

base_url="https://docs.redhat.com/en/documentation/red_hat_openshift_ai_self-managed/2.16/pdf/"
source_files=[base_url + "monitoring_data_science_models/Red_Hat_OpenShift_AI_Self-Managed-2.16-Monitoring_data_science_models-en-US.pdf",
              base_url + "monitoring_data_science_models/release_notes/Red_Hat_OpenShift_AI_Self-Managed-2.16-Release_notes-en-US.pdf",]

## Define Empty Vector Array
vectors = []

for file in source_files:
    ## Retrieve metadata from one file
    metadata = get_metadata_from_filename(get_file_name_from_url(file))
    print(metadata)
    
    ## Parse document from source chunk it
    converted_source_file = converter.convert(source_file)
    document = converted_source_file.document
    chunk_iter = chunker.chunk(document)
    ## Create chunk_list with the parts of the document
    chunk_list = list(chunk_iter)

    ## Define Empty Vector Array
    file_vectors = []

    for i, chunk in enumerate(chunk_list):
        file_vectors.append({
            "id": i, 
            "product_name": metadata.get("product_name", "null"),
            "version": metadata.get("version", "null"),
            "section": metadata.get("section", "null"),
            "language": metadata.get("language", "null"),
            "vector": embedding_model.get_text_embedding(chunk.text) , 
            "text": chunk.text,
        })

    
    vectors.append(file_vectors)
    

{'product_name': 'Red_Hat_OpenShift_AI_Self', 'version': '2.16', 'section': 'Monitoring_data_science_models', 'language': 'en'}


Token indices sequence length is longer than the specified maximum sequence length for this model (925 > 512). Running this sequence through the model will result in indexing errors


{'product_name': 'Red_Hat_OpenShift_AI_Self', 'version': '2.16', 'section': 'Release_notes', 'language': 'en'}


In [37]:
print(vectors[0])

[{'id': 0, 'vector': [-0.024205027148127556, -0.07243149727582932, -0.029754308983683586, -0.0008186743943952024, 0.045255281031131744, 0.015400868840515614, -0.0036440605763345957, 0.022229475900530815, 0.07117901742458344, 0.014413461089134216, 0.04492659121751785, -0.04016764834523201, 0.03317486494779587, 0.07123998552560806, 0.03982795029878616, 0.022307349368929863, -0.05328626558184624, 0.007630744017660618, -0.02014566957950592, -0.013482349924743176, 0.03579544648528099, -0.053175341337919235, -0.0009252880699932575, -0.04639512300491333, -0.06125780940055847, 0.019482817500829697, -0.02455763891339302, -0.05493880808353424, -0.08093786984682083, -0.15916062891483307, -0.03008163534104824, -0.053922150284051895, 0.05040591210126877, -0.0014795174356549978, 0.056870393455028534, -0.018518660217523575, -0.03644118085503578, -0.048093944787979126, -0.05249783396720886, -0.006651335395872593, -0.015019199810922146, 0.0112330736592412, -0.013540225103497505, 0.01399138942360878, 0.

## 3) Upload embeddings to Milvus

In [8]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType

## Connect to Milvus (adjust host/port as needed)
connections.connect("default", 
    host="vectordb-milvus.milvus.svc.cluster.local", 
    port="19530",
    token="root:Milvus"
)

## Define a collection schema (adjust dimensions based on your embedding size)
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=len(vectors[0]["vector"])),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65000),
    ## TODO: Add version
    #    FieldSchema(name="version", dtype=DataType.VARCHAR, max_length=1024),
]
schema = CollectionSchema(fields, description="RAG embeddings collection")

## Create or load a collection
collection_name = "rag_embeddings"
collection = Collection(name=collection_name, schema=schema)

## TODO: Create an index for faster similarity search
index_params = {"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": len(vectors[0]["vector"])}}
collection.create_index(field_name="vector", index_params=index_params)

## Load the collection for querying
collection.load()

In [9]:
## Insert vectors into Milvus
insert_output = collection.insert(vectors);

## Print to check everything is working
print(insert_output)

## Load the collection for querying
collection.load()

(insert count: 112, delete count: 0, upsert count: 0, timestamp: 455326334817665028, success count: 112, err count: 0


## 3) Query the Database to obtain RAG parsed information

In [10]:
## TODO: search
# res = client.search(
#     collection_name="my_collection",
#     data=[query_vector],
#     limit=5,
#     # highlight-start
#     filter='color like "red%" and likes > 50',
#     output_fields=["color", "likes"]
#     # highlight-end
# )

In [11]:
## TODO: 
- Creating pipeline to store more data in Vector Database

SyntaxError: invalid syntax (3796298882.py, line 2)

In [15]:
# Search data
query_vectors = embedding_model.get_text_embedding("What is Openshift AI")

res = collection.search(
    # target collection
    data=query_vectors,  # query vectors
    limit=3,
    anns_field="vector",
    param={"metric_type": "IP"},
    output_fields=["text"]
)

2025-01-15 09:30:44,428 [ERROR][handler]: RPC error: [search], <ParamError: (code=1, message=`search_data` value [0.018916407600045204, -0.06553012877702713, -0.03890419006347656, -0.000947585329413414, 0.04297740012407303, -0.007084385491907597, 0.005829900037497282, 0.006629869341850281, 0.062475334852933884, -0.03228267654776573, 0.01589363068342209, -0.02252667024731636, 0.06959574669599533, 0.06449303030967712, 0.05231038108468056, 0.030422084033489227, -0.04782497510313988, 0.019731031730771065, -0.01748519577085972, 0.0006630318239331245, 0.05464634299278259, -0.06150694936513901, -0.003995751962065697, -0.06004820391535759, -0.06094786152243614, 0.05040329322218895, -0.015113306231796741, -0.04495735466480255, -0.07351983338594437, -0.10985855013132095, -0.018881183117628098, 0.0063013723120093346, 0.04261249676346779, -0.02894873358309269, 0.043270234018564224, 0.009354321286082268, -0.041576217859983444, -0.05026301369071007, -0.07358546555042267, -0.008998801000416279, -0.00

ParamError: <ParamError: (code=1, message=`search_data` value [0.018916407600045204, -0.06553012877702713, -0.03890419006347656, -0.000947585329413414, 0.04297740012407303, -0.007084385491907597, 0.005829900037497282, 0.006629869341850281, 0.062475334852933884, -0.03228267654776573, 0.01589363068342209, -0.02252667024731636, 0.06959574669599533, 0.06449303030967712, 0.05231038108468056, 0.030422084033489227, -0.04782497510313988, 0.019731031730771065, -0.01748519577085972, 0.0006630318239331245, 0.05464634299278259, -0.06150694936513901, -0.003995751962065697, -0.06004820391535759, -0.06094786152243614, 0.05040329322218895, -0.015113306231796741, -0.04495735466480255, -0.07351983338594437, -0.10985855013132095, -0.018881183117628098, 0.0063013723120093346, 0.04261249676346779, -0.02894873358309269, 0.043270234018564224, 0.009354321286082268, -0.041576217859983444, -0.05026301369071007, -0.07358546555042267, -0.008998801000416279, -0.007698061875998974, 0.022255172953009605, -0.00439764

## 4) Query Mistral Using RAG information

In [ ]:
prompt =f"""
You are a commentator. Your task is to write a report on an essay.
When presented with the essay, come up with interesting questions to ask, and answer each question.
Afterward, combine all the information and write a report in the markdown format.

# Essay:
{markdown_dummy_2}

# Instructions:
## Summarize:
In clear and concise language, summarize the key points and themes presented in the essay.

## Interesting Questions:
Generate three distinct and thought-provoking questions that can be asked about the content of the essay. For each question:
- After "Q: ", describe the problem
- After "A: ", provide a detailed explanation of the problem addressed in the question.
- Enclose the ultimate answer in <>.

## Write a report
Using the essay summary and the answers to the interesting questions, create a comprehensive report in Markdown format.
"""

